Основаная задача - **построить хорошую тематическую модель с интерпретируемыми топиками с помощью LDA в gensim и NMF в sklearn**.

Сохраните тетрадку с экспериментами и положите её на гитхаб, ссылку на неё укажите в форме.

**Оцениваться будут главным образом пункты 5, 8 и 10. (2, 3, 2 баллов соответственно). Чтобы заработать остальные 3 балла, нужно хотя бы немного изменить мой код на промежуточных этапах (добавить что-то, указать другие параметры и т.д). **

1) сделайте нормализацию (если pymorphy2 работает долго используйте mystem или попробуйте установить быструю версию - `pip install pymorphy2[fast]`, можно использовать какой-то другой токенизатор); 

In [121]:
import gensim
import json
import re
import pandas as pd
import nltk
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
from pymorphy2 import MorphAnalyzer
import pyLDAvis.gensim
import string
from collections import Counter
import warnings
warnings.filterwarnings("ignore")

morph = MorphAnalyzer()

In [122]:
with open('habr_texts.txt', encoding='utf-8') as file:
    source = file.read().splitlines()

Решил оставить только слова написанные кирилицей, поскольку в статьях много кода, который никак не отделён от основного текста. Попытки оставить только "полезные" слова вроде названий языков программирования, фреймворков и прочего непропорционально усложняют решение, при этом, по ощущения, полезнее убрать код.

In [123]:
stop_words = set(stopwords.words('russian')) | {'gt',}

def clean_text(text):
    text = re.sub(r'<[^>]+>', '', text)
    text = re.sub(r'{[^}]+}', ' ', text)
    text = re.sub(r'\&[a-z]+\;', ' ', text)
    text = text.replace('\t', ' ')
    return text

def tokenize(text):
    return re.findall(r'[А-Яа-я]+', text)

# чтобы быстрее нормализовать тексты, создадим словарь всех словоформ
# нормазуем каждую 1 раз и положим в словарь
# затем пройдем по текстам и сопоставим каждой словоформе её нормальную форму

def opt_normalize(texts, top=None):
    uniq = Counter()
    for text in texts:
        uniq.update(text)
    
    norm_uniq = {word:morph.parse(word)[0].normal_form for word, _ in uniq.most_common(top)}
    
    norm_texts = []
    for text in texts:
        
        norm_words = [norm_uniq.get(word) for word in text]
        norm_words = [word for word in norm_words if word and word not in stop_words]
        norm_texts.append(norm_words)
        
    return norm_texts


2) добавьте нграммы (в тетрадке есть закомменченая ячейка с Phrases,  можно также попробовать другие способы построить нграммы); 

In [157]:
texts = opt_normalize([tokenize(clean_text(text)) for text in source])

In [193]:
ph = gensim.models.Phrases(texts, scoring='npmi', min_count=8, threshold=0.2)
p = gensim.models.phrases.Phraser(ph)
ngrammed_texts = p[texts]

3) сделайте хороший словарь (отфильтруйте слишком частотные и редкие слова, попробуйте удалить стоп-слова); 

In [194]:
dictionary = gensim.corpora.Dictionary(ngrammed_texts)

In [195]:
dictionary.filter_extremes(no_above=0.1, no_below=15)
dictionary.compactify()

In [196]:
corpus = [dictionary.doc2bow(text) for text in ngrammed_texts]

4) постройте несколько LDA моделей (переберите количество тем, можете поменять alpha, passes), если получаются плохие темы, поработайте дополнительно над предобработкой и словарем; 

In [202]:
lda = gensim.models.LdaMulticore(corpus, id2word=dictionary, eval_every=1, num_topics=5, passes=3, alpha='asymmetric')

In [204]:
lda.print_topics(5)

[(0,
  '0.003*"камера" + 0.002*"смартфон" + 0.002*"сигнал" + 0.002*"продажа" + 0.002*"товар" + 0.001*"услуга" + 0.001*"диск" + 0.001*"звук" + 0.001*"контроллер" + 0.001*"принтер"'),
 (1,
  '0.002*"переменный" + 0.002*"программирование" + 0.002*"курс" + 0.001*"ядро" + 0.001*"шаблон" + 0.001*"компилятор" + 0.001*"сборка" + 0.001*"протокол" + 0.001*"оператор" + 0.001*"бот"'),
 (2,
  '0.003*"телефон" + 0.002*"массив" + 0.002*"символ" + 0.002*"браузер" + 0.002*"вектор" + 0.001*"бот" + 0.001*"переменный" + 0.001*"перевод" + 0.001*"раздел" + 0.001*"книга"'),
 (3,
  '0.003*"ядро" + 0.002*"процессор" + 0.002*"робот" + 0.002*"глаз" + 0.001*"город" + 0.001*"ребёнок" + 0.001*"звук" + 0.001*"книга" + 0.001*"инструкция" + 0.001*"мозг"'),
 (4,
  '0.004*"игрок" + 0.002*"браузер" + 0.002*"ключ" + 0.001*"узел" + 0.001*"земля" + 0.001*"клетка" + 0.001*"пароль" + 0.001*"анимация" + 0.001*"движение" + 0.001*"домен"')]

In [206]:
lda = gensim.models.LdaModel(corpus, id2word=dictionary, eval_every=1, num_topics=5, passes=3, alpha='auto')

In [207]:
lda.print_topics(5)

[(0,
  '0.005*"боль" + 0.005*"домен" + 0.004*"головка" + 0.004*"инцидент" + 0.004*"печать" + 0.004*"пароль" + 0.004*"атака" + 0.003*"уязвимость" + 0.003*"клетка" + 0.003*"принтер"'),
 (1,
  '0.005*"браузер" + 0.004*"массив" + 0.004*"контекст" + 0.003*"шаблон" + 0.003*"атрибут" + 0.003*"бот" + 0.003*"переменный" + 0.003*"логика" + 0.003*"кампания" + 0.003*"контейнер"'),
 (2,
  '0.008*"игрок" + 0.004*"студент" + 0.003*"мозг" + 0.003*"играть" + 0.003*"дом" + 0.003*"звук" + 0.003*"доклад" + 0.003*"курс" + 0.002*"электроника" + 0.002*"аудитория"'),
 (3,
  '0.005*"вакансия" + 0.004*"ядро" + 0.003*"яркость" + 0.003*"стандарт" + 0.003*"сигнал" + 0.003*"дефект" + 0.003*"процессор" + 0.003*"цикл" + 0.003*"узел" + 0.003*"диск"'),
 (4,
  '0.008*"услуга" + 0.005*"заказчик" + 0.002*"продажа" + 0.002*"российский" + 0.002*"трафик" + 0.002*"предприятие" + 0.002*"публикация" + 0.002*"реклама" + 0.002*"китай" + 0.002*"инвестиция"')]

5) для самой хорошей модели в отдельной ячейке напечатайте 3 хороших (на ваш вкус) темы;

6) между словарем и обучением модели добавьте tfidf (`gensim.models.TfidfModel(corpus, id2word=dictionary); corpus = tfidf[corpus]`);

In [209]:
tfidf = gensim.models.TfidfModel(corpus, id2word=dictionary)
corpus = tfidf[corpus]

7) повторите пункт 4 на преобразованном корпусе;

In [215]:
lda = gensim.models.LdaMulticore(corpus, id2word=dictionary, eval_every=1, num_topics=5, passes=3, alpha='asymmetric')

In [216]:
lda.print_topics(5)

[(0,
  '0.001*"игрок" + 0.001*"браузер" + 0.001*"доклад" + 0.001*"ключ" + 0.001*"бот" + 0.001*"камера" + 0.001*"ядро" + 0.001*"смартфон" + 0.001*"книга" + 0.001*"диск"'),
 (1,
  '0.001*"бот" + 0.001*"книга" + 0.000*"телефон" + 0.000*"робот" + 0.000*"камера" + 0.000*"дом" + 0.000*"сколько_часы" + 0.000*"браузер" + 0.000*"фотография" + 0.000*"стартап"'),
 (2,
  '0.000*"игрок" + 0.000*"кампания" + 0.000*"ядро" + 0.000*"письмо" + 0.000*"бот" + 0.000*"мониторинг" + 0.000*"блокировка" + 0.000*"хакатон" + 0.000*"рецепт" + 0.000*"рис"'),
 (3,
  '0.000*"врач" + 0.000*"блокировка" + 0.000*"сообщение_ошибка" + 0.000*"браузер" + 0.000*"бэкап" + 0.000*"лекция" + 0.000*"дефект" + 0.000*"домен" + 0.000*"менеджер" + 0.000*"р"'),
 (4,
  '0.000*"плагин" + 0.000*"локализация" + 0.000*"профиль" + 0.000*"родной_язык" + 0.000*"флаг" + 0.000*"панель" + 0.000*"станок" + 0.000*"шапка" + 0.000*"синтезатор" + 0.000*"социальный_сеть"')]

In [217]:
lda = gensim.models.LdaMulticore(corpus, id2word=dictionary, eval_every=1, num_topics=5, passes=3, alpha='asymmetric')

In [218]:
lda.print_topics(5)

[(0,
  '0.001*"игрок" + 0.001*"браузер" + 0.001*"бот" + 0.001*"доклад" + 0.001*"ядро" + 0.001*"ключ" + 0.001*"книга" + 0.001*"камера" + 0.001*"смартфон" + 0.001*"диск"'),
 (1,
  '0.000*"письмо" + 0.000*"камера" + 0.000*"книга" + 0.000*"бот" + 0.000*"драйвер" + 0.000*"игрок" + 0.000*"подложка" + 0.000*"ключ" + 0.000*"переменный" + 0.000*"робот"'),
 (2,
  '0.000*"доклад" + 0.000*"игрок" + 0.000*"вм" + 0.000*"никак_влиять" + 0.000*"звук" + 0.000*"высокий_вероятность" + 0.000*"хост" + 0.000*"сборка" + 0.000*"кластер" + 0.000*"виртуальный_машина"'),
 (3,
  '0.000*"вселенная" + 0.000*"инфляция" + 0.000*"биткоина" + 0.000*"рабочий_стол" + 0.000*"дерево" + 0.000*"сегментация" + 0.000*"вебинар" + 0.000*"всплывать_окно" + 0.000*"скидка" + 0.000*"энергия"'),
 (4,
  '0.000*"собеседование" + 0.000*"сокет" + 0.000*"симулятор" + 0.000*"ядро" + 0.000*"метрика" + 0.000*"ндс" + 0.000*"папка" + 0.000*"мастер" + 0.000*"робот" + 0.000*"вектор"')]

8) в отдельной ячейке опишите как изменилась модель (приведите несколько тем, которые стали лучше или хуже, или которых раньше вообще не было; можно привести значения перплексии и когерентности для обеих моделей)

9) проделайте такие же действия для NMF (образец в конце тетрадки), для построения словаря воспользуйтесь возможностями Count или Tfidf Vectorizer (попробуйте другие значение max_features, min_df, max_df, сделайте нграмы через ngram_range, если хватает памяти), попробуйте такие же количества тем

10) в отдельной ячейки напечатайте темы лучшей NMF модели, сравните их с теми, что получились в LDA.